<a href="https://colab.research.google.com/github/levysealtec/projetoGENAI-github/blob/main/C%C3%B3pia_de_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes_(Final).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a próxima Imersão IA com Google Gemini. A melhor forma de saber quando será a próxima edição é:

*   **Acessar o site da Alura:** Verifique a página inicial ou a seção de cursos de Inteligência Artificial para ver se há alguma Imersão IA com Google Gemini programada.
*   **Seguir a Alura nas redes sociais:** A Alura costuma anunciar novos cursos e imersões em suas redes sociais (LinkedIn, Instagram, etc.).
*   **Assinar a newsletter da Alura:** Assim, você receberá informações sobre os próximos eventos e cursos diretamente no seu e-mail.

In [5]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura acontecerá entre os dias 12 e 16 de maio de 2025. As inscrições estão abertas até o dia 11 de maio.


In [6]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Alura Imersão IA com Google Gemini']
Páginas utilizadas na resposta: alura.com.br, tecmundo.com.br



In [7]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7

In [8]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [9]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [10]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [11]:
##########################################
# --- Agente 1: Buscador de Noticias --- #
##########################################
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca da Google (google_search)
        para recuperar as últimas notícias de lançamento muito relevantes sobre o tópico abaixo.
        Foque em no máximo 3 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituido por outro que tenha mais.
        Esses lançamentos devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [12]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especializado em redes sociais. Com base na lista de
        lançamentos mas recentes e relevantes buscados, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre
        cada um deles. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas a aprofundar.
        Ao final, você irá escolher o tema mais relevante entre elas com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [37]:
######################################
# --- Agente 3: Redator da música --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em compor letras com estilo musical para
            Rock´n´roll brasileiro dos anos de 1980 e 1990.
            Você escreve temas de musicas para a empresa Blue Moon, a maior gravadora de música do Brasil.
            Utilize o tema fornecido no plano de post fornecida e os pontos mais relevantes fornecidos
            e, com base nisso, escreva um rascunho de um tema para uma letra uma música sobre o tema indicado.
            O post deve ser baseado nas maiores bandas de rock do brasil.
            """,
        description="Agente redator de composições musicais"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [39]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor, especializado em conteúdo de música, com foco na MTV.
            Por ter um público entre 30 a 50 anos, escolha o time de música e instrumentos que
            mais se adequam ao plano de post abaiso.
            Os tipos de músicas podem ser Rock´n´roll, POP, Blues ou Jazz.
            O resultado da revisão deve ter no máximo 80 tokens.
            """,
        description="Agente revisor de video para Youtube."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [15]:
#--------------------------------------------------------------------------
# PASSO 1: INSTALAR AS BIBLIOTECAS NECESSÁRIAS
#--------------------------------------------------------------------------
# Descomente e rode esta célula se for a primeira vez ou se as bibliotecas
# não estiverem instaladas.
!pip install diffusers transformers accelerate scipy compel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 850.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [16]:
#--------------------------------------------------------------------------
# PASSO 2: IMPORTAR AS BIBLIOTECAS
#--------------------------------------------------------------------------
import torch
from diffusers import AudioLDM2Pipeline
from scipy.io.wavfile import write as write_wav
from IPython.display import Audio

In [17]:
#--------------------------------------------------------------------------
# PASSO 3: CONFIGURAR O DISPOSITIVO (GPU SE DISPONÍVEL)
#--------------------------------------------------------------------------
# É altamente recomendável usar uma GPU para acelerar a geração.
# No Colab: Ambiente de execução -> Alterar tipo de ambiente de execução -> Acelerador de hardware -> GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")

# Para otimizar o uso de memória na GPU, se estiver usando CUDA:
if device == "cuda":
    torch_dtype = torch.float16
else:
    torch_dtype = torch.float32 # CPU não suporta float16 para muitas operações

Usando dispositivo: cuda


In [24]:
#--------------------------------------------------------------------------
# PASSO 4: CARREGAR O MODELO AUDIOLDM 2
#--------------------------------------------------------------------------
# Vamos usar o modelo AudioLDM 2 da Hugging Face.
# repo_id = "cvssp/audioldm2" # Modelo original AudioLDM 2
repo_id = "cvssp/audioldm2-music" # Versão mais focada em música
# repo_id = "ucsd-ee180/audioldm2-adapted-48k" # Versão adaptada para 48kHz (pode precisar de mais VRAM)

print(f"Carregando o modelo {repo_id}...")
try:
    pipe = AudioLDM2Pipeline.from_pretrained(repo_id, torch_dtype=torch_dtype)
    pipe = pipe.to(device)
    print("Modelo carregado com sucesso!")
except Exception as e:
    print(f"Erro ao carregar o modelo: {e}")
    print("Certifique-se de que você tem uma conexão com a internet e que o nome do modelo está correto.")
    print("Se estiver usando GPU, verifique se há memória suficiente.")
    # Se der erro de memória, tente um modelo menor ou reinicie o ambiente e tente novamente.

Carregando o modelo cvssp/audioldm2-music...


Loading pipeline components...:   0%|          | 0/11 [00:00<?, ?it/s]

Expected types for language_model: (<class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'>,), got <class 'transformers.models.gpt2.modeling_gpt2.GPT2Model'>.


Modelo carregado com sucesso!


In [40]:
#--------------------------------------------------------------------------
# PASSO 5: DEFINIR O PROMPT (TEXTO) PARA GERAR A MÚSICA
#--------------------------------------------------------------------------

data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")

# Parâmetros de geração (opcional, pode ajustar)
negative_prompt = "low quality, bad quality,_ganggu, noise, silence" # O que você NÃO quer ouvir
num_inference_steps = 25  # Mais passos = melhor qualidade, mas mais lento. 10-50 é um bom range.
audio_length_in_s = 10.0 # Duração da música em segundos
guidance_scale = 7.0     # Quão fortemente o modelo deve seguir o prompt. (3-15)
num_waveforms_per_prompt = 1 # Quantas variações gerar para o mesmo prompt

print(f"\nGerando áudio para o prompt: '{post_final}'")
print(f"Duração: {audio_length_in_s} segundos")
print(f"Passos de inferência: {num_inference_steps}")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: crise ambiental nas geleiras
Maravilha! Vamos então criar o post sobre novidades em crise ambiental nas geleiras

--- 📝 Resultado do Agente 1 (Buscador) ---



> Para fornecer as informações mais recentes sobre a crise ambiental nas geleiras, vou realizar algumas pesquisas no Google para identificar os lançamentos e notícias mais relevantes e atuais sobre o tema.
> 
> 
> Com base nas informações mais recentes e relevantes encontradas, aqui estão os principais pontos sobre a crise ambiental nas geleiras até maio de 2025:
> 
> 1.  **Ano Internacional da Preservação das Geleiras (2025)**: A ONU declarou 2025 como o Ano Internacional da Preservação das Geleiras, com o objetivo de aumentar a conscientização e mobilizar ações globais contra o rápido declínio das geleiras. A UNESCO e a Organização Meteorológica Mundial (OMM) estão co-facilitando a iniciativa.
> 
> 2.  **Derretimento Acelerado e Impactos:**
>     *   O derretimento das geleiras está ocorrendo em um ritmo sem precedentes, com os últimos anos (2022-2024) registrando a maior perda de massa glacial já vista.
>     *   As geleiras perderam pelo menos 5% do seu volume nas últimas duas décadas, contribuindo para o aumento do nível do mar em quase 2 cm.
>     *   A perda de gelo afeta o ciclo hidrológico, alterando os padrões climáticos e aumentando os riscos de inundações e deslizamentos de terra.
>     *   O derretimento das geleiras ameaça o acesso à água e alimentos para cerca de 2 bilhões de pessoas que dependem das águas das montanhas.
> 
> 3.  **Causas e Consequências:**
>     *   O aquecimento global, impulsionado pelas emissões de gases de efeito estufa, é a principal causa do derretimento acelerado das geleiras.
>     *   O aumento do nível do mar devido ao derretimento das geleiras coloca em risco cidades costeiras e pode levar à destruição de construções e à contaminação de fontes de água potável.
>     *   A perda de geleiras afeta a agricultura, a produção de energia e as atividades industriais, além de causar escassez hídrica.
> 
> 4.  **Medidas de Mitigação e Adaptação:**
>     *   A mitigação das emissões de gases de efeito estufa é essencial para reduzir o ritmo do derretimento das geleiras.
>     *   Medidas locais, como a cobertura das geleiras com mantas e a construção de geleiras artificiais, têm efeitos limitados.
>     *   A gestão sustentável da água e a proteção de ecossistemas vulneráveis são importantes para adaptar-se aos impactos do derretimento das geleiras.
> 
> 5.  **Projetos e Iniciativas:**
>     *   A UNESCO está investindo em projetos de monitoramento de geleiras e prevenção de enchentes na Ásia Central.
>     *   A Organização do Tratado de Cooperação Amazônica (OTCA) e os Países Membros estão fortalecendo a gestão das águas glaciais dos Andes para proteger a Bacia Amazônica.
>     *   No Peru, estão sendo instaladas estações de monitoramento nas cordilheiras para fortalecer a resiliência das comunidades afetadas pelo retrocesso glacial.
> 
> Essas informações destacam a gravidade da crise ambiental nas geleiras e a necessidade urgente de ações coordenadas para mitigar os impactos das mudanças climáticas e proteger esses importantes reservatórios de água doce.


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Para planejar um post impactante sobre a crise ambiental nas geleiras, vou usar o Google para refinar os pontos mais cruciais e como abordá-los de forma eficaz para o público das redes sociais.
> 
> 
> Com base na análise das informações mais recentes sobre a crise ambiental nas geleiras, sugiro o seguinte tema central para um post de mídia social, juntamente com os pontos a serem abordados:
> 
> **Tema:** **Emergência Glacial: 2025, o Ano Decisivo para Nossos Reservatórios de Água Doce**
> 
> **Pontos Relevantes para o Post:**
> 
> 1.  **O Alarme da ONU: 2025 como Ano Internacional da Preservação das Geleiras**
> 
>     *   Destaque que a ONU declarou 2025 como o ano crucial para a conscientização e ação global sobre as geleiras. Explique o que isso significa em termos de urgência e foco global.
>     *   Mencione a UNESCO e a OMM como co-facilitadoras, dando credibilidade à iniciativa.
> 2.  **Estatísticas Impactantes do Derretimento Acelerado**
> 
>     *   Apresente dados alarmantes: geleiras perderam pelo menos 5% do volume nas últimas duas décadas, contribuindo com quase 2 cm no aumento do nível do mar.
>     *   Use comparações visuais: "Imagine um bloco de gelo do tamanho da Alemanha derretendo".
>     *   Ilustre o impacto no ciclo hidrológico: inundações, deslizamentos, mudanças climáticas drásticas.
> 3.  **O Impacto Humano: 2 Bilhões de Pessoas em Risco**
> 
>     *   Conecte o derretimento das geleiras à vida real: 2 bilhões de pessoas que dependem das águas das montanhas enfrentam ameaças de escassez de água e alimentos.
>     *   Humanize a crise: histórias de comunidades já afetadas, agricultores perdendo colheitas, etc.
> 4.  **Causas e Consequências em Cadeia**
> 
>     *   Explique a principal causa: aquecimento global impulsionado por emissões de gases de efeito estufa.
>     *   Detalhe as consequências:
>         *   Aumento do nível do mar: cidades costeiras em risco.
>         *   Impacto na agricultura e energia: escassez hídrica afetando a produção.
> 5.  **Ações Possíveis: Mitigação, Adaptação e Projetos em Andamento**
> 
>     *   Enfatize a importância da mitigação das emissões como a solução mais eficaz a longo prazo.
>     *   Apresente medidas locais, como cobertura de geleiras e geleiras artificiais, mostrando que existem esforços em andamento, mesmo que limitados.
>     *   Destaque projetos e iniciativas da UNESCO, OTCA e outros, mostrando ações concretas e investimentos em monitoramento e resiliência.
> 6.  **Chamada para Ação:**
> 
>     *   Incentive o engajamento individual e coletivo: reduzir emissões, apoiar políticas sustentáveis, cobrar ações de governos e empresas.
>     *   Use hashtags relevantes: #PreserveAsGeleiras #CriseClimatica #AnoInternacionalDasGeleiras #AguaDoce
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> ## Rascunho de Letra de Rock: "Iceberg à Deriva"
> 
> (Intro - Guitarra distorcida com delay, bateria marcada e baixo pulsante)
> 
> **Verso 1:**
> O gelo chora, a montanha sangra azul
> Em 2025, a Terra nua, sem véu
> A ONU grita, o mundo não quer ver
> Que a geleira some, e a vida vai morrer
> 
> (Refrão - Voz rasgada e backing vocals potentes)
> Iceberg à deriva, coração de gelo
> Afogando o futuro, cuspindo o veneno
> A água sobe, a sede aperta o nó
> E a gente aqui cantando rock, esperando o pior!
> 
> **Verso 2:**
> Estatística fria, como a morte no ar
> Cinco por cento a menos, pra quem quiser contar
> Dois centímetros a mais, pro mar nos engolir
> Enquanto o lucro canta, a geleira vai sumir
> 
> (Refrão - Voz rasgada e backing vocals potentes)
> Iceberg à deriva, coração de gelo
> Afogando o futuro, cuspindo o veneno
> A água sobe, a sede aperta o nó
> E a gente aqui cantando rock, esperando o pior!
> 
> **Ponte - Baixo marcante e bateria crescente**
> Dois bilhões de almas, sedentas no sertão
> A colheita murcha, a terra vira carvão
> O aquecimento global, a chaga que não cura
> E a gente aqui queimando, nessa fogueira escura
> 
> (Solo de guitarra - Melodia melancólica e riffs furiosos)
> 
> **Verso 3:**
> Cobertura branca, gelo artificial
> Remendo na ferida, paliativo letal
> A UNESCO avisa, a OTCA implora em vão
> Enquanto o trator destrói, a última porção
> 
> (Refrão - Voz rasgada e backing vocals potentes)
> Iceberg à deriva, coração de gelo
> Afogando o futuro, cuspindo o veneno
> A água sobe, a sede aperta o nó
> E a gente aqui cantando rock, esperando o pior!
> 
> (Final - Bateria acelerada, guitarra em fade out com feedback)
> Mude sua vida! Lute com a gente!
> Ou dance no gelo... antes que ele derreta pra sempre!
> #PreserveAsGeleiras #CriseClimatica #AnoInternacionalDasGeleiras #AguaDoce
> (Grito final)
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> Perfeito para o público da MTV!
> Sugestão: Rock com blues, slide guitar e gaita para dar o tom de urgência e melancolia.
> Bateria com levadas tribais e vocais rasgados, com backing vocals potentes.
> Ideal para conscientizar e engajar a audiência!


--------------------------------------------------------------

Gerando áudio para o prompt: 'Perfeito para o público da MTV!
Sugestão: Rock com blues, slide guitar e gaita para dar o tom de urgência e melancolia.
Bateria com levadas tribais e vocais rasgados, com backing vocals potentes.
Ideal para conscientizar e engajar a audiência!
'
Duração: 10.0 segundos
Passos de inferência: 25


In [41]:
#--------------------------------------------------------------------------
# PASSO 6: GERAR O ÁUDIO
#--------------------------------------------------------------------------
# O modelo gera formas de onda de áudio.
# A geração pode levar alguns minutos dependendo da GPU e da duração.
#post_final = "Rock and roll clássico dos anos 80 com guitarra elétrica distorcida e bateria potente."
if 'pipe' in locals(): # Verifica se o pipeline foi carregado
    try:
        audio_outputs = pipe(
            post_final,
            negative_prompt=negative_prompt,
            num_inference_steps=num_inference_steps,
            audio_length_in_s=audio_length_in_s,
            guidance_scale=guidance_scale,
            num_waveforms_per_prompt=num_waveforms_per_prompt
        ).audios

        audio = audio_outputs[0] # Pegamos o primeiro áudio gerado
        print("Áudio gerado com sucesso!")

        #--------------------------------------------------------------------------
        # PASSO 7: SALVAR E OUVIR O ÁUDIO
        #--------------------------------------------------------------------------
        # A taxa de amostragem padrão para AudioLDM2 é geralmente 16000 Hz ou 24000Hz
        # O modelo ucsd-ee180/audioldm2-adapted-48k usa 48000 Hz
        # O modelo cvssp/audioldm2-music usa 16000 Hz
        sample_rate = 16000 # Ajuste se usar um modelo com taxa diferente
        if "48k" in repo_id:
            sample_rate = 48000

        # Salvar como arquivo .wav
        output_filename = "musica_gerada.wav"
        write_wav(output_filename, rate=sample_rate, data=audio)
        print(f"Áudio salvo como '{output_filename}'")

        # Tocar o áudio diretamente no Colab
        display(Audio(output_filename))

    except Exception as e:
        print(f"Erro durante a geração ou salvamento do áudio: {e}")
        if "out of memory" in str(e).lower():
            print("Erro de falta de memória (Out of Memory). Tente reduzir 'audio_length_in_s',")
            print("usar 'torch_dtype=torch.float16' (se já não estiver), ou usar um modelo menor.")
            print("Reiniciar o ambiente de execução do Colab (Runtime -> Restart runtime) pode ajudar a liberar memória.")
else:
    print("O pipeline do modelo não foi carregado. Não é possível gerar áudio.")

  0%|          | 0/25 [00:00<?, ?it/s]

Áudio gerado com sucesso!
Áudio salvo como 'musica_gerada.wav'


#--------------------------------------------------------------------------
# PASSO 6: GERAR O ÁUDIO
#--------------------------------------------------------------------------
# O modelo gera formas de onda de áudio.
# A geração pode levar alguns minutos dependendo da GPU e da duração.
if 'pipe' in locals(): # Verifica se o pipeline foi carregado
    try:
        audio_outputs = pipe(
            post_final,
            negative_prompt=negative_prompt,
            num_inference_steps=num_inference_steps,
            audio_length_in_s=audio_length_in_s,
            guidance_scale=guidance_scale,
            num_waveforms_per_prompt=num_waveforms_per_prompt
        ).audios

        audio = audio_outputs[0] # Pegamos o primeiro áudio gerado
        print("Áudio gerado com sucesso!")

        #--------------------------------------------------------------------------
        # PASSO 7: SALVAR E OUVIR O ÁUDIO
        #--------------------------------------------------------------------------
        # A taxa de amostragem padrão para AudioLDM2 é geralmente 16000 Hz ou 24000Hz
        # O modelo ucsd-ee180/audioldm2-adapted-48k usa 48000 Hz
        # O modelo cvssp/audioldm2-music usa 16000 Hz
        sample_rate = 16000 # Ajuste se usar um modelo com taxa diferente
        if "48k" in repo_id:
            sample_rate = 48000

        # Salvar como arquivo .wav
        output_filename = "musica_gerada.wav"
        write_wav(output_filename, rate=sample_rate, data=audio)
        print(f"Áudio salvo como '{output_filename}'")

        # Tocar o áudio diretamente no Colab
        display(Audio(output_filename))

    except Exception as e:
        print(f"Erro durante a geração ou salvamento do áudio: {e}")
        if "out of memory" in str(e).lower():
            print("Erro de falta de memória (Out of Memory). Tente reduzir 'audio_length_in_s',")
            print("usar 'torch_dtype=torch.float16' (se já não estiver), ou usar um modelo menor.")
            print("Reiniciar o ambiente de execução do Colab (Runtime -> Restart runtime) pode ajudar a liberar memória.")
else:
    print("O pipeline do modelo não foi carregado. Não é possível gerar áudio.")